In [98]:
import numpy as np
import os
import cv2
from numpy.lib.stride_tricks import sliding_window_view
import warnings
import tensorflow as tf
from tensorflow import keras

CONVOLUTION:

In [99]:
def batch_convolution(data, filters, stride, padding='valid'):
    """
    Optimized batch convolution operation with 4D input support
    """
    # Input validation code remains unchanged
        # Input validation
    if not isinstance(data, np.ndarray) or data.ndim != 4:
        raise ValueError("Input data must be 4D numpy array [B, C, H, W]")
    
    if not isinstance(filters, np.ndarray) or filters.ndim != 4:
        raise ValueError("Filters must be 4D numpy array [N, C, FH, FW]")
    
    if filters.shape[1] != data.shape[1]:
        raise ValueError(f"Filter channels ({filters.shape[1]}) must match data channels ({data.shape[1]})")
    
    batch_size, in_channels, in_h, in_w = data.shape
    num_filters, _, fh, fw = filters.shape
    sh, sw = stride
    batch_size, in_channels, in_h, in_w = data.shape
    num_filters, _, fh, fw = filters.shape
    sh, sw = stride
    
    # Calculate output dimensions remains unchanged
    if padding == 'same':
        i=0
        # Same padding calculation code
        # ...
    elif padding == 'valid':
        out_h = (in_h - fh) // sh + 1
        out_w = (in_w - fw) // sw + 1
    else:
        raise ValueError("Padding must be 'valid' or 'same'")

    # Create sliding windows for each sample in the batch
    windows = sliding_window_view(data, (1, 1, fh, fw), axis=(0, 1, 2, 3))
    
    # Apply stride
    windows = windows[:, :, ::sh, ::sw, 0, :, :, :]
    
    # FIXED: Get actual window shape rather than trying to unpack
    window_shape = windows.shape
    
    # FIXED: Reshape using actual dimensions
    # Flatten batch, channels, and spatial dimensions
    windows_reshaped = windows.reshape(-1, in_channels * fh * fw)
    
    # Reshape filters to match inner dimension
    filters_reshaped = filters.reshape(num_filters, in_channels * fh * fw)
    
    # Matrix multiplication with matching dimensions
    output = np.matmul(windows_reshaped, filters_reshaped.T)
    
    # Reshape output to proper dimensions
    output = output.reshape(batch_size, out_h, out_w, num_filters)
    output = output.transpose(0, 3, 1, 2)  # NHWC -> NCHW
    
    return output


DECONVOLUTION:

In [100]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

def batch_deconvolution(data, filters, stride, padding='valid', output_padding=(0, 0)):
    """
    Vectorized transposed convolution (deconvolution) with batch support

    Parameters:
    data : np.ndarray [batch_size, in_channels, in_height, in_width]
    filters : np.ndarray [out_channels, in_channels, fh, fw]
    stride : tuple (sh, sw)
    padding : 'valid' or 'same'
    output_padding : tuple (oph, opw)

    Returns:
    output : np.ndarray [batch_size, out_channels, out_height, out_width]
    """
    if data.ndim != 4 or filters.ndim != 4:
        raise ValueError("Inputs must be 4D arrays")

    batch_size, in_chan, in_h, in_w = data.shape
    out_chan, _, fh, fw = filters.shape
    sh, sw = stride
    oph, opw = output_padding

    # Calculate expected output shape
    if padding == 'same':
        out_h = in_h * sh + oph
        out_w = in_w * sw + opw
        pad_h = (in_h - 1) * sh + fh - out_h + oph
        pad_w = (in_w - 1) * sw + fw - out_w + opw
        pad_h = max(pad_h, 0)
        pad_w = max(pad_w, 0)
    elif padding == 'valid':
        out_h = (in_h - 1) * sh + fh + oph
        out_w = (in_w - 1) * sw + fw + opw
        pad_h = pad_w = 0
    else:
        raise ValueError("Padding must be 'valid' or 'same'")

    # Step 1: Upsample
    up_h = (in_h - 1) * sh + 1
    up_w = (in_w - 1) * sw + 1
    upsampled = np.zeros((batch_size, in_chan, up_h, up_w), dtype=data.dtype)
    upsampled[:, :, ::sh, ::sw] = data

    # Step 2: Pad
    pad_top = pad_h // 2
    pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2
    pad_right = pad_w - pad_left
    padded = np.pad(
        upsampled,
        [(0, 0), (0, 0), (pad_top, pad_bottom), (pad_left, pad_right)],
        mode='constant'
    )

    # Step 3: Adjust padding if needed to match output shape
    needed_h = out_h + fh - 1
    needed_w = out_w + fw - 1
    cur_h, cur_w = padded.shape[2], padded.shape[3]
    extra_pad_h = max(0, needed_h - cur_h)
    extra_pad_w = max(0, needed_w - cur_w)

    if extra_pad_h > 0 or extra_pad_w > 0:
        padded = np.pad(
            padded,
            [(0, 0), (0, 0), (0, extra_pad_h), (0, extra_pad_w)],
            mode='constant'
        )

    # Step 4: Sliding window view
    windows = sliding_window_view(padded, (fh, fw), axis=(2, 3))  # shape: [B, C, H_out, W_out, fh, fw]

    # Step 5: Apply filters with einsum
    output = np.einsum('bcijxy,ocxy->boij', windows, filters)

    # Final sanity check
    assert output.shape[2] == out_h and output.shape[3] == out_w, \
        f"Deconv output shape mismatch: got {output.shape[2:]}, expected ({out_h}, {out_w})"
    
    return output


RELU:

In [101]:
def relu(data):
    return np.maximum(0.0,data)

BATCH NORMALIZATION:

In [102]:
def batch_norm(data, bn, eps=1e-5):

    gamma=bn[0,:]
    beta=bn[1,:]
    """  
    data : np.ndarray [N, C, H, W]  
    gamma: np.ndarray [C,]  
    beta : np.ndarray [C,]  
    """  
    # Per-channel statistics  
    mu = np.mean(data, axis=(0, 2, 3), keepdims=True)  
    var = np.var(data, axis=(0, 2, 3), keepdims=True)  

    # Normalization  
    x_hat = (data - mu) / np.sqrt(var + eps)  

    # Scale and shift  
    return gamma[None,:,None,None] * x_hat + beta[None,:,None,None]  

DERIVATIVE FOR BATCH NORMALIZATION:

In [103]:
def batch_norm_backward(dout, x, gamma, eps=1e-5):  
    mu = np.mean(x, axis=(0,2,3), keepdims=True)  
    var = np.var(x, axis=(0,2,3), keepdims=True)  
    x_hat = (x - mu)/np.sqrt(var + eps)  

    N, C, H, W = x.shape  
    dgamma = np.sum(dout * x_hat, axis=(0,2,3))  
    dbeta = np.sum(dout, axis=(0,2,3))  

    dx_hat = dout * gamma[None,:,None,None]  
    dvar = np.sum(dx_hat * (x - mu) * (-0.5) * (var + eps)**(-1.5), axis=(0,2,3))  
    dmu = np.sum(dx_hat * (-1)/np.sqrt(var + eps), axis=(0,2,3)) + dvar * np.mean(-2*(x - mu), axis=(0,2,3))  

    dx = dx_hat / np.sqrt(var + eps) + dvar[None,:,None,None]*2*(x - mu)/N/H/W + dmu[None,:,None,None]/N/H/W  
    dbn=np.zeros((2,gamma.shape[0]))
    dbn[0,:]=dgamma
    dbn[1,:]=dbeta
    return dx, dbn  


DERIVATIVE FOR RELU:

In [104]:
def relu_backward(data):
    return (data>0).astype(float)

DERIVATIVE FOR CONVOLUTION:

In [105]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

def conv_backward(dout, x, W, stride, padding='valid'):
    """
    Vectorized backward pass for batch convolution.
    """
    batch_size, in_chan, h_in, w_in = x.shape
    num_filters, _, fh, fw = W.shape
    sh, sw = stride

    # === Padding for x (input) ===
    if padding == 'same':
        out_h = int(np.ceil(h_in / sh))
        out_w = int(np.ceil(w_in / sw))
        pad_h = max((out_h - 1) * sh + fh - h_in, 0)
        pad_w = max((out_w - 1) * sw + fw - w_in, 0)
        pad_top = pad_h // 2
        pad_bottom = pad_h - pad_top
        pad_left = pad_w // 2
        pad_right = pad_w - pad_left
        x_padded = np.pad(x, [(0,0), (0,0), (pad_top, pad_bottom), (pad_left, pad_right)])
    else:
        x_padded = x
        out_h = (h_in - fh) // sh + 1
        out_w = (w_in - fw) // sw + 1

    # === dW: Vectorized filter gradient ===
    # Extract input patches as in the forward pass
    x_windows = sliding_window_view(x_padded, (fh, fw), axis=(2, 3))
    x_windows = x_windows[:, :, ::sh, ::sw, :, :]  # Apply stride
    # x_windows shape: (B, in_chan, out_h, out_w, fh, fw)
    # dout shape: (B, num_filters, out_h, out_w)
    # We want: (num_filters, in_chan, fh, fw)
    dW = np.einsum('b o h w, b c h w f g -> o c f g', dout, x_windows)

    # === dx: Vectorized input gradient ===
    # For dx, we need to convolve dout with flipped filters
    W_flip = np.flip(W, axis=(2, 3)).swapaxes(0,1)  # (in_chan, num_filters, fh, fw)
    # Upsample dout to account for stride
    H_out, W_out = dout.shape[2], dout.shape[3]
    H_upsampled = (H_out - 1) * sh + 1
    W_upsampled = (W_out - 1) * sw + 1
    dout_upsampled = np.zeros((batch_size, num_filters, H_upsampled, W_upsampled), dtype=dout.dtype)
    dout_upsampled[:, :, ::sh, ::sw] = dout

    # Pad dout_upsampled for full convolution
    pad_h = fh - 1
    pad_w = fw - 1
    dout_padded = np.pad(dout_upsampled, [(0,0), (0,0), (pad_h, pad_h), (pad_w, pad_w)])

    # Extract sliding windows from padded dout
    dout_windows = sliding_window_view(dout_padded, (fh, fw), axis=(2,3))
    # dout_windows shape: (B, num_filters, h_in, w_in, fh, fw)
    # W_flip shape: (in_chan, num_filters, fh, fw)
    # We want: (B, in_chan, h_in, w_in)
    dx = np.einsum('b o h w f g, c o f g -> b c h w', dout_windows, W_flip)

    # Remove padding from dx if 'same'
    if padding == 'same':
        dx = dx[:, :, :h_in, :w_in]

    return dx, dW


DERIVATIVE FOR DECONVOLUTION:

In [106]:
from scipy.signal import correlate2d

In [107]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from scipy.signal import correlate2d

def deconv_backward(dout, x, W, stride, padding='valid', output_padding=(0,0)):
    B, in_C, H_in, W_in = x.shape
    out_C, _, fh, fw = W.shape
    sh, sw = stride
    oph, opw = output_padding

    # Gradient w.r.t. input (dx)
    W_flipped = np.flip(W, axis=(2,3)).transpose(1, 0, 2, 3)  # [in_C, out_C, fh, fw]

    if padding == 'same':
        pad_h = fh - 1
        pad_w = fw - 1
    elif padding == 'valid':
        pad_h = fh - 1 + oph
        pad_w = fw - 1 + opw
    else:
        raise ValueError("Padding must be 'valid' or 'same'")

    pad_top = pad_h // 2
    pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2
    pad_right = pad_w - pad_left

    dout_padded = np.pad(dout, [(0,0), (0,0), (pad_top, pad_bottom), (pad_left, pad_right)], mode='constant')

    dx = np.zeros((B, in_C, H_in, W_in))
    for b in range(B):
        for i in range(in_C):
            for o in range(out_C):
                conv_result = correlate2d(dout[b, o], W_flipped[i, o], mode='valid')
                dx[b, i] += conv_result[::sh, ::sw]

    # Gradient w.r.t. weights (dW)
    # Calculate required padded size
    H_up = (H_in - 1) * sh + 1
    W_up = (W_in - 1) * sw + 1
    x_upsampled = np.zeros((B, in_C, H_up, W_up))
    x_upsampled[:, :, ::sh, ::sw] = x    
    required_h = dout.shape[2] + fh - 1
    required_w = dout.shape[3] + fw - 1

    cur_h, cur_w = x_upsampled.shape[2], x_upsampled.shape[3]
    pad_h = required_h - cur_h
    pad_w = required_w - cur_w

    pad_top = pad_h // 2
    pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2
    pad_right = pad_w - pad_left

    if pad_h > 0 or pad_w > 0:
        x_upsampled = np.pad(
            x_upsampled,
            [(0, 0), (0, 0), (pad_top, pad_bottom), (pad_left, pad_right)],
            mode='constant'
        )

    x_windows = sliding_window_view(x_upsampled, (fh, fw), axis=(2, 3))  # [B, in_C, H_out, W_out, fh, fw]

    H_out, W_out = dout.shape[2], dout.shape[3]
    if x_windows.shape[2] != H_out or x_windows.shape[3] != W_out:
        x_windows = x_windows[:, :, :H_out, :W_out, :, :]

    dW = np.einsum('bohw,bchwxy->ocxy', dout, x_windows)

    return dx, dW


ERROR METRICS:

In [108]:
def hybrid_loss(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError()(y_true, y_pred)
    vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet')
    perceptual_model = tf.keras.Model(vgg.input, vgg.get_layer('block3_conv3').output)
    pl = tf.reduce_mean(tf.square(perceptual_model(y_true) - perceptual_model(y_pred)))
    return 0.7*pl + 0.3*mse

In [109]:
def square_error(IP,OP):
    error=(IP-OP)**2
    return error

MODEL:

In [110]:
dataset_path="D:\REQS\PROJECT\dataset\\train"

#Hypreparameters
epochs=10
back_spin=5
learning_rate=0.0001
e=0.00001
batch_size=32
image_size=[3,32,32]

In [111]:
#MODEL:2 -- THE BASE

s_all=[2,2]
#3x32x32
conv_f1=np.random.randn(8,3,4,4)* np.sqrt(2/(3*4*4))
bn_f1=np.random.randn(2,8)* np.sqrt(2/(3*4*4))
#8x15x15
conv_f2=np.random.randn(12,8,3,3)* np.sqrt(2/(3*4*4))
bn_f2=np.random.rand(2,12)* np.sqrt(2/(3*4*4))
#12x7x7
deconv_f3=np.random.randn(10,12,5,5)* np.sqrt(2/(3*4*4))
bn_f3=np.random.rand(2,10)* np.sqrt(2/(3*4*4))
#10x17x17
deconv_f4=np.random.randn(6,10,4,4)* np.sqrt(2/(3*4*4))
bn_f4=np.random.rand(2,6)* np.sqrt(2/(3*4*4))
#6x36x36
deconv_f5=np.random.randn(3,6,5,5)* np.sqrt(2/(3*4*4))
bn_f5=np.random.rand(2,3)* np.sqrt(2/(3*4*4))
#3x75x75
conv_f6=np.random.randn(6,3,5,5)* np.sqrt(2/(3*4*4))
bn_f6=np.random.rand(2,6)* np.sqrt(2/(3*4*4))
#6x36x36
conv_f7=np.random.randn(10,6,4,4)* np.sqrt(2/(3*4*4))
bn_f7=np.random.rand(2,10)* np.sqrt(2/(3*4*4))
#10x17x17
conv_f8=np.random.randn(12,10,5,5)* np.sqrt(2/(3*4*4))
bn_f8=np.random.rand(2,12)* np.sqrt(2/(3*4*4))
#12x7x7
deconv_f9=np.random.randn(8,12,3,3)* np.sqrt(2/(3*4*4))
bn_f9=np.random.rand(2,8)* np.sqrt(2/(3*4*4))
#8x15x15
deconv_f10=np.random.randn(3,8,4,4)* np.sqrt(2/(3*4*4))
bn_f10=np.random.rand(2,3)* np.sqrt(2/(3*4*4))
#3x32x32

In [112]:
#flow 1,2,3,4,5,6,7,8,9,10
def forward(IP):
    a1=batch_convolution(IP,conv_f1,(2,2),'valid')
    bn_a1=batch_norm(a1, bn_f1,0.00001)
    act_a1=relu(bn_a1)
    #print(act_a1.shape)
    a2=batch_convolution(act_a1,conv_f2,(2,2),'valid',)
    bn_a2=batch_norm(a2, bn_f2,0.00001)
    act_a2=relu(bn_a2)
    #print(act_a2.shape)
    a3=batch_deconvolution(act_a2,deconv_f3,(2,2),'valid',(0,0))
    bn_a3=batch_norm(a3, bn_f3,0.00001)
    act_a3=relu(bn_a3)
    #print(act_a3.shape)
    a4=batch_deconvolution(act_a3,deconv_f4,(2,2),'valid',(0,0))
    bn_a4=batch_norm(a4, bn_f4,0.00001)
    act_a4=relu(bn_a4)
    #print(act_a4.shape)
    a5=batch_deconvolution(act_a4,deconv_f5,(2,2),'valid',(0,0))
    bn_a5=batch_norm(a5, bn_f5,0.00001)
    act_a5=relu(bn_a5)
    #print(act_a5.shape)
    a6=batch_convolution(act_a5,conv_f6,(2,2),'valid')
    bn_a6=batch_norm(a6, bn_f6,0.00001)
    act_a6=relu(bn_a6)
    #print(act_a6.shape)
    a7=batch_convolution(act_a6,conv_f7,(2,2),'valid')
    bn_a7=batch_norm(a7, bn_f7,0.00001)
    act_a7=relu(bn_a7)
    #print(act_a7.shape)
    a8=batch_convolution(act_a7,conv_f8,(2,2),'valid')
    bn_a8=batch_norm(a8, bn_f8,0.00001)
    act_a8=relu(bn_a8)
    a9=batch_deconvolution(act_a8,deconv_f9,(2,2),'valid',(0,0))
    bn_a9=batch_norm(a9, bn_f9,0.00001)
    act_a9=relu(bn_a9)
    a10=batch_deconvolution(act_a9,deconv_f10,(2,2),'valid',(0,0))
    bn_a10=batch_norm(a10, bn_f10,0.00001)
    act_a10=relu(bn_a10)
    
    return act_a10, bn_a10, a10, act_a9, bn_a9, a9, act_a8, bn_a8, a8, act_a7, bn_a7, a7, act_a6, bn_a6, a6, act_a5, bn_a5, a5, act_a4, bn_a4, a4, act_a3, bn_a3, a3, act_a2, bn_a2, a2, act_a1, bn_a1, a1

In [113]:
def m1_backward(IP, layers, losse, saver):
    act_a10, bn_a10, a10, act_a9, bn_a9, a9, act_a8, bn_a8, a8, act_a7, bn_a7, a7, act_a6, bn_a6, a6, act_a5, bn_a5, a5, act_a4, bn_a4, a4, act_a3, bn_a3, a3, act_a2, bn_a2, a2, act_a1, bn_a1, a1=layers
    d_act_a10=losse
    d_bn_a10=relu_backward(d_act_a10)
    d_a10, d_bn_f10=batch_norm_backward(d_bn_a10, a10, bn_f10[0,:])
    d_act_a9, d_deconv_f10=deconv_backward(d_a10, act_a9, deconv_f10, (2,2))
    #print(d_act_a9.shape)
    d_bn_a9=relu_backward(d_act_a9)
    d_a9, d_bn_f9=batch_norm_backward(d_bn_a9, a9, bn_f9[0,:])
    d_act_a8, d_deconv_f9=deconv_backward(d_a9, act_a8, deconv_f9, (2,2))
    #print(d_act_a8.shape)
    d_bn_a8=relu_backward(d_act_a8)
    d_a8, d_bn_f8=batch_norm_backward(d_bn_a8, a8, bn_f8[0,:])
    d_act_a7, d_conv_f8=conv_backward(d_a8, act_a7, conv_f8, (2,2))
    #print(d_act_a7.shape)
    d_bn_a7=relu_backward(d_act_a7)
    d_a7, d_bn_f7=batch_norm_backward(d_bn_a7, a7, bn_f7[0,:])
    d_act_a6, d_conv_f7=conv_backward(d_a7, act_a6, conv_f7, (2,2))
    #print(d_act_a6.shape)
    d_bn_a6=relu_backward(d_act_a6)
    d_a6, d_bn_f6=batch_norm_backward(d_bn_a6, a6, bn_f6[0,:])
    d_act_a5, d_conv_f6=conv_backward(d_a6, act_a5, conv_f6, (2,2))
    #print(d_act_a5.shape)
    d_bn_a5=relu_backward(d_act_a5)
    d_a5, d_bn_f5=batch_norm_backward(d_bn_a5, a5, bn_f5[0,:])
    d_act_a4, d_deconv_f5=deconv_backward(d_a5, act_a4, deconv_f5, (2,2))
    #print(d_act_a4.shape)
    d_bn_a4=relu_backward(d_act_a4)
    d_a4, d_bn_f4=batch_norm_backward(d_bn_a4, a4, bn_f4[0,:])
    d_act_a3, d_deconv_f4=deconv_backward(d_a4, act_a3, deconv_f4, (2,2))
    #print(d_act_a3.shape)
    d_bn_a3=relu_backward(d_act_a3)
    d_a3, d_bn_f3=batch_norm_backward(d_bn_a3, a3, bn_f3[0,:])
    d_act_a2, d_deconv_f3=deconv_backward(d_a3, act_a2, deconv_f3, (2,2))
    #print(d_act_a2.shape)
    d_bn_a2=relu_backward(d_act_a2)
    d_a2, d_bn_f2=batch_norm_backward(d_bn_a2, a2, bn_f2[0,:])
    d_act_a1, d_conv_f2=conv_backward(d_a2, act_a1, conv_f2, (2,2))
    #print(d_act_a1.shape)
    d_bn_a1=relu_backward(d_act_a1)
    d_a1, d_bn_f1=batch_norm_backward(d_bn_a1, a1, bn_f1[0,:])
    d_IP, d_conv_f1=conv_backward(d_a1, IP, conv_f1, (2,2))
    saver=d_IP
    return d_conv_f1, d_bn_f1, d_conv_f2, d_bn_f2, d_deconv_f3, d_bn_f3, d_deconv_f4, d_bn_f4, d_deconv_f5, d_bn_f5, d_conv_f6, d_bn_f6, d_conv_f7, d_bn_f7, d_conv_f8, d_bn_f8, d_deconv_f9, d_bn_f9, d_deconv_f10, d_bn_f10

In [114]:
#Update Parameters
def update_params(diffs, lr, conv_f1, conv_f2, deconv_f3, deconv_f4, deconv_f5, conv_f6, conv_f7, conv_f8, deconv_f9, deconv_f10, bn_f1, bn_f2, bn_f3, bn_f4, bn_f5, bn_f6, bn_f7, bn_f8, bn_f9, bn_f10):
    d_conv_f1, d_bn_f1, d_conv_f2, d_bn_f2, d_deconv_f3, d_bn_f3, d_deconv_f4, d_bn_f4, d_deconv_f5, d_bn_f5, d_conv_f6, d_bn_f6, d_conv_f7, d_bn_f7, d_conv_f8, d_bn_f8, d_deconv_f9, d_bn_f9, d_deconv_f10, d_bn_f10=diffs
    conv_f1=conv_f1-lr*d_conv_f1
    bn_f1-=lr*d_bn_f1
    conv_f2-=lr*d_conv_f2
    bn_f2-=lr*d_bn_f2
    deconv_f3-=lr*d_deconv_f3
    bn_f3-=lr*d_bn_f3
    deconv_f4-=lr*d_deconv_f4
    bn_f4-=lr*d_bn_f4
    deconv_f5-=lr*d_deconv_f5
    bn_f5-=lr*d_bn_f5
    conv_f6-=lr*d_conv_f6
    bn_f6-=lr*d_bn_f6
    conv_f7-=lr*d_conv_f7
    bn_f7-=lr*d_bn_f7
    conv_f8-=lr*d_conv_f8
    bn_f8-=lr*d_bn_f8
    deconv_f9-=lr*d_deconv_f9
    bn_f9-=lr*d_bn_f9
    deconv_f10-=lr*d_deconv_f10
    bn_f10-=lr*d_bn_f10
    return

In [115]:
def load_image_paths():
    """Loads all image file paths from the dataset directory."""
    supported_formats = ('.png', '.jpg', '.jpeg', '.bmp')
    image_paths = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) 
                    if f.endswith(supported_formats)]
    return image_paths
    
def load_image(image_path):
    """Loads an image, resizes it, and normalizes pixel values to the range [0, 1]."""
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not load image: {image_path}")
    #Eimage = cv2.resize(image, image_size)
    image = image.astype(np.float64) / 255.0  # Normalize to [0, 1]
    return image


In [117]:
def get_batch(image_paths):
    """Generates a batch of images for training."""
    batch_paths = np.random.choice(image_paths, batch_size, replace=False)
    images = np.array([load_image(path) for path in batch_paths])
    return images

prev= np.zeros((batch_size, 3,32,32))    
image_paths = load_image_paths()
for epoch in range(epochs):
    total_loss = 0
    num_batches = len(image_paths) // batch_size
        
    for batch_index in range(num_batches):
        batch = get_batch(image_paths)
        batch = np.transpose(batch, (0,3,1,2))  # Proper channel-first format
        # Forward pass through the model
        results = forward(batch)
        act_a10, bn_a10, a10, act_a9, bn_a9, a9, act_a8, bn_a8, a8, act_a7, bn_a7, a7, act_a6, bn_a6, a6, act_a5, bn_a5, a5, act_a4, bn_a4, a4, act_a3, bn_a3, a3, act_a2, bn_a2, a2, act_a1, bn_a1, a1=results
        saver=np.zeros((batch.shape))    
        # Compute error (assume the error function is already implemented in the model)
        losse = square_error(act_a10, batch)
        loss =np.mean(losse)
        # Backward pass to compute gradients
        box=m1_backward(batch, results, 0.7*losse+ 0.3*prev, saver)
        prev =saver
        # Update model parameters (assume the model has a method to update its parameters)
        update_params(box, learning_rate, conv_f1, conv_f2, deconv_f3, deconv_f4, deconv_f5, conv_f6, conv_f7, conv_f8, deconv_f9, deconv_f10, bn_f1, bn_f2, bn_f3, bn_f4, bn_f5, bn_f6, bn_f7, bn_f8, bn_f9, bn_f10)
        
        total_loss += loss

        if (batch_index + 1) % 2 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_index+1}/{num_batches}], Loss: {loss:.4f}")
        
    avg_loss = total_loss / num_batches
    print(f"Epoch [{epoch+1}/{epochs}] completed. Average Loss: {avg_loss:.4f}")

Epoch [1/10], Batch [2/1562], Loss: 0.2850
Epoch [1/10], Batch [4/1562], Loss: 0.2932
Epoch [1/10], Batch [6/1562], Loss: 0.3113
Epoch [1/10], Batch [8/1562], Loss: 0.2913
Epoch [1/10], Batch [10/1562], Loss: 0.2892
Epoch [1/10], Batch [12/1562], Loss: 0.2961
Epoch [1/10], Batch [14/1562], Loss: 0.2737
Epoch [1/10], Batch [16/1562], Loss: 0.2781
Epoch [1/10], Batch [18/1562], Loss: 0.3085
Epoch [1/10], Batch [20/1562], Loss: 0.2674
Epoch [1/10], Batch [22/1562], Loss: 0.2592
Epoch [1/10], Batch [24/1562], Loss: 0.3401
Epoch [1/10], Batch [26/1562], Loss: 0.3010
Epoch [1/10], Batch [28/1562], Loss: 0.2736
Epoch [1/10], Batch [30/1562], Loss: 0.3049
Epoch [1/10], Batch [32/1562], Loss: 0.3151
Epoch [1/10], Batch [34/1562], Loss: 0.2831
Epoch [1/10], Batch [36/1562], Loss: 0.3154
Epoch [1/10], Batch [38/1562], Loss: 0.2710
Epoch [1/10], Batch [40/1562], Loss: 0.3034
Epoch [1/10], Batch [42/1562], Loss: 0.2881
Epoch [1/10], Batch [44/1562], Loss: 0.3470
Epoch [1/10], Batch [46/1562], Loss:

KeyboardInterrupt: 

In [ ]:
params = [
    ('conv_f1', conv_f1), ('bn_f1', bn_f1),
    ('conv_f2', conv_f2), ('bn_f2', bn_f2),
    ('deconv_f3', deconv_f3), ('bn_f3', bn_f3),
    ('deconv_f4', deconv_f4), ('bn_f4', bn_f4),
    ('deconv_f5', deconv_f5), ('bn_f5', bn_f5),
    ('conv_f6', conv_f6), ('bn_f6', bn_f6),
    ('conv_f7', conv_f7), ('bn_f7', bn_f7),
    ('conv_f8', conv_f8), ('bn_f8', bn_f8),
    ('deconv_f9', deconv_f9), ('bn_f9', bn_f9),
    ('deconv_f10', deconv_f10), ('bn_f10', bn_f10)
]

print(",\n".join([f"{name}: {value}" for name, value in params]))


conv_f1: [[[[-2.07815755e-01  3.24387129e-02 -7.23291664e-02  7.59978048e-02]
   [-4.31037945e-01 -4.35561155e-02 -4.30622177e-02 -1.86366234e-01]
   [ 1.68575944e-01  1.27352658e-01  1.16083010e-01 -3.68937533e-01]
   [-2.77735084e-01 -1.68362683e-01 -1.79665512e-01 -1.45318130e-01]]

  [[-2.69944423e-01 -3.16140522e-01 -1.92722061e-01 -8.80538576e-04]
   [-3.81507178e-01 -8.83259037e-02  3.73870026e-02 -8.69346871e-02]
   [-3.05138210e-01 -2.71642536e-01 -1.69896979e-01  8.50410066e-02]
   [-1.18933094e-01  1.26259258e-01  3.92716285e-01 -5.82326861e-02]]

  [[-2.13520248e-01 -2.27311900e-01  2.29037262e-01 -1.30004643e-01]
   [ 1.08292054e-01  3.74445097e-01 -2.04655138e-01  4.10167807e-01]
   [-1.59604890e-02 -1.75636055e-01 -1.89652690e-01 -2.35240242e-01]
   [-4.08108460e-01  7.61471768e-02 -1.49836401e-01  2.92420095e-01]]]


 [[[-3.09836165e-01 -5.51054572e-02 -2.33975563e-01  8.84351820e-03]
   [ 3.08575698e-01  2.39634054e-02 -1.98955732e-01  3.84344480e-01]
   [-1.07003771e-